In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms,models
from PIL import Image
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

# Load JSON configuration
config_json = """
{
    "device": "cuda",
    "dataset_path": "/content/drive/MyDrive/archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    "disease_directories": [
  "Apple___Apple_scab",
  "Apple___Black_rot",
  "Apple___Cedar_apple_rust",
  "Blueberry___healthy",
  "Cherry_(including_sour)___Powdery_mildew",
  "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
  "Corn_(maize)___Common_rust_",
  "Corn_(maize)___Northern_Leaf_Blight",
  "Grape___Black_rot",
  "Grape___Esca_(Black_Measles)",
  "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
  "Orange___Haunglongbing_(Citrus_greening)",
  "Peach___Bacterial_spot",
  "Pepper,_bell___Bacterial_spot",
  "Potato___Early_blight",
  "Potato___Late_blight",
  "Raspberry___healthy",
  "Soybean___healthy",
  "Squash___Powdery_mildew",
  "Strawberry___Leaf_scorch",
  "Tomato___Spider_mites Two-spotted_spider_mite",
  "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
  "Tomato___Tomato_mosaic_virus"
]
,
    "annotations": [
  {
    "filename": "Apple___Apple_scab",
    "disease": "Apple Scab",
    "symptoms": "Circular, olive-green to dark brown lesions on leaves, fruit, and young twigs. Leaves may become distorted, curl, or drop prematurely. Infected fruit develops rough, scaly patches and may crack. Severe infections can cause defoliation, reducing fruit yield and quality.",
    "treatment": "Apply fungicides like captan, mancozeb, or myclobutanil early in the season, especially before rainy periods. Use resistant apple varieties such as 'Enterprise', 'Liberty', and 'Freedom'. Remove fallen leaves and infected fruit to prevent overwintering of the fungus. Prune trees to improve air circulation and reduce moisture buildup. Avoid overhead watering and ensure proper spacing between trees for better air circulation."
  },
  {
    "filename": "Apple___Black_rot",
    "disease": "Black Rot (Apple)",
    "symptoms": "Small, circular brown spots on leaves that enlarge and develop a reddish-purple border. Infected fruit shows concentric rings, turns black, and shrivels into mummified fruit. Cankers form on branches, appearing as sunken, dark lesions with cracked bark.",
    "treatment": "Prune and destroy infected branches and mummified fruit to prevent the spread of the fungus. Apply copper-based or sulfur fungicides during early bloom and after petal fall. Ensure good air circulation by proper pruning and spacing of trees. Avoid overhead watering to reduce moisture levels. Maintain orchard sanitation by removing fallen leaves and fruit."
  },
  {
    "filename": "Apple___Cedar_apple_rust",
    "disease": "Cedar Apple Rust",
    "symptoms": "Bright orange, gelatinous spore masses appear on the undersides of leaves. Small yellow spots develop on the upper leaf surface, expanding into orange-red lesions. Infected fruit may show similar spots and become deformed. Severe infections can cause early leaf drop and reduced fruit yield.",
    "treatment": "Remove or prune nearby cedar and juniper trees to break the disease cycle. Apply protective fungicides like myclobutanil, mancozeb, or propiconazole during early spring and throughout the growing season. Plant resistant apple varieties such as 'Liberty' and 'Enterprise'. Ensure good air circulation by proper pruning and avoid overhead watering to reduce leaf wetness."
}
,
  {
    "filename": "Blueberry___healthy",
    "disease": "Healthy (Blueberry)",
    "symptoms": "No disease symptoms. Leaves are green and firm, with no discoloration, spots, or wilting. Stems are strong, and fruit develops normally without signs of rot or mold.",
    "treatment": "Maintain good agricultural practices, including proper irrigation, balanced fertilization, and regular pruning to promote air circulation. Monitor plants for early signs of disease or pest infestations. Ensure well-drained soil with appropriate pH (4.5–5.5) to support healthy growth. Mulch around plants to retain moisture and suppress weeds."
}
,
  {
    "filename": "Cherry_(including_sour)___Powdery_mildew",
    "disease": "Powdery Mildew (Cherry)",
    "symptoms": "White, powdery fungal growth appears on the upper and lower leaf surfaces. Leaves may curl, become distorted, and show yellowing. In severe cases, buds, young shoots, and fruit can also be affected, leading to stunted growth and reduced yield.",
    "treatment": "Apply sulfur-based fungicides or potassium bicarbonate sprays at the first sign of infection. Prune affected branches and remove fallen leaves to prevent the spread of spores. Ensure proper spacing and pruning to improve air circulation. Avoid excessive nitrogen fertilization, which can encourage mildew growth. Water plants at the base to keep foliage dry and reduce humidity levels."
}
,
  {
    "filename": "Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot",
    "disease": "Cercospora Leaf Spot (Corn)",
    "symptoms": "Small, grayish-brown rectangular lesions with purple to reddish-brown borders appear on lower leaves first. Lesions may enlarge and merge, leading to extensive leaf blight. Severe infections reduce photosynthesis, causing premature leaf death and lower yield.",
    "treatment": "Rotate crops with non-host plants like soybeans or wheat to reduce fungal spore buildup. Use resistant or tolerant corn varieties for better disease management. Apply fungicides such as strobilurins or triazoles during early infection stages. Improve field drainage and ensure proper plant spacing to reduce humidity and promote airflow. Remove crop residues after harvest to minimize fungal overwintering."
}
,
  {
    "filename": "Corn_(maize)___Common_rust_",
    "disease": "Common Rust (Corn)",
    "symptoms": "Small, reddish-brown pustules appear on both leaf surfaces, often surrounded by a yellow halo. As the disease progresses, pustules darken and merge, leading to leaf blight and reduced photosynthesis. Severe infections can weaken plants and lower yield.",
    "treatment": "Plant resistant corn hybrids to minimize infection risk. Apply fungicides such as strobilurins or triazoles when disease pressure is high. Ensure proper plant spacing and improve air circulation to reduce moisture buildup. Avoid excessive nitrogen fertilization, which can make plants more susceptible."
}
,
  {
    "filename": "Corn_(maize)___Northern_Leaf_Blight",
    "disease": "Northern Leaf Blight (Corn)",
    "symptoms": "Long, cigar-shaped gray-green lesions develop on lower leaves first and expand as the disease progresses. Lesions turn tan and may develop dark fungal spores under humid conditions. Severe infections cause leaf blight, reducing grain fill and yield.",
    "treatment": "Use resistant corn hybrids to prevent infections. Apply fungicides like azoxystrobin or propiconazole during early symptom development. Rotate crops with non-host plants to break the disease cycle. Improve field drainage and plant spacing to reduce humidity and promote air circulation."
}
,
  {
    "filename": "Grape___Black_rot",
    "disease": "Black Rot (Grape)",
    "symptoms": "Small, circular brown spots with darker borders appear on leaves, eventually enlarging and turning necrotic. Infected fruit develops sunken, dark brown lesions before shriveling into hard, black mummies. The disease spreads rapidly in warm, humid conditions.",
    "treatment": "Prune and destroy infected vines, leaves, and fruit to prevent fungal spread. Apply fungicides such as myclobutanil, mancozeb, or captan during early growth stages. Ensure proper air circulation by pruning and trellising. Avoid overhead watering and remove mummified berries to reduce fungal overwintering."
}
,
  {
    "filename": "Grape___Esca_(Black_Measles)",
    "disease": "Esca (Black Measles, Grape)",
    "symptoms": "Dark, irregular streaks appear on the woody tissues of vines. Leaves show interveinal necrosis with a tiger-stripe pattern of yellow and brown areas. Affected fruit may develop sunken, discolored spots and shrivel prematurely. Severe cases lead to vine dieback.",
    "treatment": "Avoid wounding vines during pruning to prevent fungal entry. Remove and destroy infected wood and prune symptomatic areas. Maintain vine health with balanced fertilization and proper irrigation. Improve vineyard drainage and minimize stress factors to reduce disease susceptibility."
}
,
  {
    "filename": "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "disease": "Leaf Blight (Grape)",
    "symptoms": "Irregular brown lesions develop on leaves, often with a yellow halo. Severely infected leaves curl, dry out, and drop prematurely. The disease weakens vines and reduces fruit production.",
    "treatment": "Prune and remove affected leaves and branches to improve air circulation. Apply fungicides such as mancozeb or copper-based sprays during early infection stages. Ensure proper vineyard sanitation and avoid overhead irrigation to minimize moisture on leaves."
}
,
  {
    "filename": "Orange___Haunglongbing_(Citrus_greening)",
    "disease": "Citrus Greening (Huanglongbing)",
    "symptoms": "Leaves exhibit asymmetrical yellowing with blotchy mottling. Fruits remain small, misshapen, and have a bitter taste. Infected trees show stunted growth, twig dieback, and premature fruit drop.",
    "treatment": "Control psyllid insect vectors using insecticides and biological control methods. Remove and destroy infected trees to prevent disease spread. Use disease-free nursery stock for new plantings and implement strict quarantine measures in affected areas."
}
,
  {
    "filename": "Peach___Bacterial_spot",
    "disease": "Bacterial Spot (Peach)",
    "symptoms": "Small, dark, water-soaked lesions form on leaves, which later turn reddish-brown with a shot-hole appearance. Fruit develops sunken, dark spots that may ooze and crack, reducing market quality.",
    "treatment": "Apply copper-based sprays or bactericides like streptomycin during early growth stages. Avoid overhead irrigation to reduce bacterial spread. Plant resistant peach varieties and practice crop rotation to minimize recurrence."
}
,
  {
    "filename": "Pepper,_bell___Bacterial_spot",
    "disease": "Bacterial Spot (Pepper)",
    "symptoms": "Water-soaked lesions appear on leaves, enlarging into dark brown spots with yellow halos. Severe infections cause leaf drop, stunted growth, and blemished fruit, reducing yield.",
    "treatment": "Use certified disease-free seeds and resistant varieties. Apply copper-based bactericides to manage early infections. Avoid working in wet fields to prevent bacterial spread and use proper crop rotation techniques."
}
,
  {
    "filename": "Potato___Early_blight",
    "disease": "Early Blight (Potato)",
    "symptoms": "Dark, concentric rings form on older leaves, resembling a target pattern. Leaves turn yellow and wither as the disease progresses. Severe infections reduce tuber size and yield.",
    "treatment": "Plant resistant potato varieties and ensure proper crop rotation. Apply fungicides such as chlorothalonil or azoxystrobin at the first sign of infection. Maintain good field sanitation and remove infected plant debris after harvest."
}
,
  {
    "filename": "Potato___Late_blight",
    "disease": "Late Blight (Potato)",
    "symptoms": "Dark, water-soaked lesions appear on leaves, rapidly expanding under humid conditions. A white, fuzzy mold develops on the undersides of infected leaves. Stems and tubers may also show dark, rotting areas, leading to plant collapse and severe yield loss.",
    "treatment": "Apply fungicides like chlorothalonil, mancozeb, or metalaxyl at the first sign of infection. Avoid overhead irrigation to reduce leaf wetness. Ensure proper field drainage and remove infected plants to prevent disease spread. Rotate crops and use disease-free seed potatoes."
}
,
  {
    "filename": "Raspberry___healthy",
    "disease": "Healthy (Raspberry)",
    "symptoms": "No disease symptoms. Leaves are green and firm, with no spots, wilting, or discoloration. Canes are strong and productive, producing healthy fruit.",
    "treatment": "Maintain proper crop management, including pruning for good air circulation. Monitor for pests and diseases regularly. Use well-drained soil with proper irrigation and mulching to retain moisture and suppress weeds. Fertilize appropriately for optimal growth."
}
,
  {
    "filename": "Soybean___healthy",
    "disease": "Healthy (Soybean)",
    "symptoms": "No disease symptoms. Leaves are green and fully developed without signs of spots, yellowing, or curling. Plants grow uniformly and produce healthy pods.",
    "treatment": "Use proper agricultural practices, including crop rotation, balanced fertilization, and timely irrigation. Monitor for pests and diseases regularly. Plant high-quality, disease-resistant soybean varieties to maintain healthy crops."
}
,
  {
    "filename": "Squash___Powdery_mildew",
    "disease": "Powdery Mildew (Squash)",
    "symptoms": "White, powdery fungal growth appears on the upper surfaces of leaves, stems, and sometimes fruit. Leaves may curl, yellow, and eventually dry out, reducing plant vigor and yield.",
    "treatment": "Apply sulfur-based fungicides or potassium bicarbonate sprays at the first sign of infection. Improve air circulation by properly spacing plants and pruning excess foliage. Avoid overhead watering and plant resistant squash varieties when available."
}
,
  {
    "filename": "Strawberry___Leaf_scorch",
    "disease": "Leaf Scorch (Strawberry)",
    "symptoms": "Small, reddish-brown spots with yellow halos appear on leaves. As the disease progresses, spots enlarge and merge, causing leaf edges to dry out and curl. Severe infections reduce plant vigor and fruit production.",
    "treatment": "Improve air circulation by proper plant spacing and pruning. Apply fungicides such as copper-based sprays or myclobutanil at early symptom stages. Water at the base of plants to keep foliage dry and reduce fungal spread. Remove and destroy infected leaves to limit disease progression."
}
,
  {
    "filename": "Tomato___Spider_mites_Two-spotted_spider_mite",
    "disease": "Spider Mites (Two-Spotted Spider Mite, Tomato)",
    "symptoms": "Tiny yellow stippling appears on leaves as mites feed on plant cells. Fine webbing is visible on the undersides of leaves and between stems. Severe infestations cause leaf bronzing, curling, and defoliation, leading to reduced plant vigor and yield.",
    "treatment": "Use miticides such as abamectin or spiromesifen for effective control. Introduce predatory mites like Phytoseiulus persimilis to naturally reduce populations. Spray plants with neem oil or insecticidal soap to suppress infestations. Maintain proper irrigation to reduce plant stress, which makes them more susceptible to mites."
}
,
  {
    "filename": "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "disease": "Tomato Yellow Leaf Curl Virus",
    "symptoms": "Young leaves curl upwards, become thickened, and turn yellow. Plants exhibit stunted growth and reduced fruit production. Flowers may drop prematurely, leading to poor yields.",
    "treatment": "Control whiteflies, the primary vectors, using insecticides or neem oil. Introduce beneficial insects like ladybugs to reduce whitefly populations. Plant virus-resistant tomato varieties to minimize infection risk. Use physical barriers such as insect-proof netting to prevent whitefly infestation."
}
,
  {
    "filename": "Tomato___Tomato_mosaic_virus",
    "disease": "Tomato Mosaic Virus",
    "symptoms": "Leaves develop a mottled or mosaic pattern of light and dark green areas. Plants exhibit stunted growth, distorted leaves, and reduced fruit yield. Infected fruits may show discoloration or uneven ripening.",
    "treatment": "Use virus-free seeds and resistant tomato varieties. Remove and destroy infected plants to prevent the spread. Disinfect tools and hands after handling plants to reduce mechanical transmission. Control aphids and other potential insect vectors that may spread the virus."
}

]
,
    "train_batch_size": 28,
    "num_epochs": 20,
    "learning_rate": 0.001
}
"""
# Parse JSON configuration
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
disease_directories = config["disease_directories"]

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the CNN model for plant disease classification
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Load trained CNN model
num_classes = 27
cnn_model = SimpleCNN(num_classes).to(device)
model_path = "/content/drive/MyDrive/plant_disease_cnn_model.h5"
cnn_model.load_state_dict(torch.load(model_path))
cnn_model.eval()
print("Plant disease model loaded successfully.")

# Load chatbot model (e.g., FLAN-T5, GPT, etc.)
chatbot_model_id = "/content/drive/MyDrive/lora-flan-t5-large-chat"  # Replace with your chatbot model
tokenizer = AutoTokenizer.from_pretrained(chatbot_model_id)
chatbot_model = AutoModelForSeq2SeqLM.from_pretrained(chatbot_model_id).to(device)
print("Chatbot model loaded successfully.")

# Function to predict the disease
def predict_disease(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = cnn_model(image)
            _, predicted = torch.max(outputs, 1)
        return disease_directories[predicted.item()]
    except Exception as e:
        return f"Error processing image: {e}"

# Function to generate chatbot response
def get_chatbot_response(disease, query):
    prompt = f"The plant has {disease}. {query}"
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(device)
    outputs = chatbot_model.generate(input_ids=input_ids, max_length=200, do_sample=True, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Interactive chatbot
print("AI Assistant: Hello! I can help you identify plant diseases and answer related queries.")
current_disease = None

while True:
    if not current_disease:
        image_path = input("AI Assistant: Please provide the path to a plant image: ")
        if not os.path.exists(image_path):
            print("AI Assistant: Invalid path. Please try again.")
            continue

        predicted_disease = predict_disease(image_path)
        if "Error" in predicted_disease:
            print(f"AI Assistant: {predicted_disease}")
            continue

        current_disease = predicted_disease
        print(f"AI Assistant: The plant seems to have {current_disease}.")
        print("AI Assistant: You can ask about symptoms, treatments, or any other details. Type 'new' to analyze a new image.")
        continue

    user_query = input("You: ").strip().lower()
    if user_query == "new":
        current_disease = None
        print("AI Assistant: Let's analyze a new image!")
        continue

    response = get_chatbot_response(current_disease, user_query)
    print(f"AI Assistant: {response}")


<ipython-input-7-8853b3067de5>:233: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_model.load_state_dict(torch.load(model_path))


Plant disease model loaded successfully.
Chatbot model loaded successfully.
AI Assistant: Hello! I can help you identify plant diseases and answer related queries.
AI Assistant: Please provide the path to a plant image: /content/drive/MyDrive/archive/test/test/TomatoHealthy1.JPG
AI Assistant: The plant seems to have Apple___Apple_scab.
AI Assistant: You can ask about symptoms, treatments, or any other details. Type 'new' to analyze a new image.
You: new
AI Assistant: Let's analyze a new image!
AI Assistant: Please provide the path to a plant image: /content/drive/MyDrive/archive/test/test/AppleScab1.JPG
AI Assistant: The plant seems to have Apple___Apple_scab.
AI Assistant: You can ask about symptoms, treatments, or any other details. Type 'new' to analyze a new image.


In [ ]:
/content/drive/MyDrive/chatbot.py

In [ ]:
script_path = "/content/drive/MyDrive/chatbot.py"

code = """\
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

# Load JSON configuration
config_json = '''
{
    "device": "cuda",
    "dataset_path": "/content/drive/MyDrive/archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    "disease_directories": [
        "Apple___Apple_scab",
        "Apple___Black_rot",
        "Apple___Cedar_apple_rust",
        "Blueberry___healthy",
        "Cherry_(including_sour)___Powdery_mildew",
        "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
        "Corn_(maize)___Common_rust_",
        "Corn_(maize)___Northern_Leaf_Blight",
        "Grape___Black_rot",
        "Grape___Esca_(Black_Measles)",
        "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
        "Orange___Haunglongbing_(Citrus_greening)",
        "Peach___Bacterial_spot",
        "Pepper,_bell___Bacterial_spot",
        "Potato___Early_blight",
        "Potato___Late_blight",
        "Raspberry___healthy",
        "Soybean___healthy",
        "Squash___Powdery_mildew",
        "Strawberry___Leaf_scorch",
        "Tomato___Spider_mites Two-spotted_spider_mite",
        "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
        "Tomato___Tomato_mosaic_virus"
    ],
    "train_batch_size": 28,
    "num_epochs": 20,
    "learning_rate": 0.001
}
'''
config = json.loads(config_json)
device = config["device"] if torch.cuda.is_available() else "cpu"
disease_directories = config["disease_directories"]

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load CNN model for plant disease classification
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Load trained CNN model
num_classes = 27
cnn_model = SimpleCNN(num_classes).to(device)
model_path = "/content/drive/MyDrive/plant_disease_cnn_model.pth"
cnn_model.load_state_dict(torch.load(model_path, map_location=device))
cnn_model.eval()
print("Plant disease model loaded successfully.")

# Load chatbot model (FLAN-T5)
chatbot_model_id = "/content/drive/MyDrive/lora-flan-t5-large-chat"
tokenizer = AutoTokenizer.from_pretrained(chatbot_model_id)
chatbot_model = AutoModelForSeq2SeqLM.from_pretrained(chatbot_model_id).to(device)
print("Chatbot model loaded successfully.")

# Function to predict the disease
def predict_disease(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = cnn_model(image)
            _, predicted = torch.max(outputs, 1)
        return disease_directories[predicted.item()]
    except Exception as e:
        return f"Error processing image: {e}"

# Function to generate chatbot response
def get_chatbot_response(disease, query):
    prompt = f"The plant has {disease}. {query}"
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(device)
    outputs = chatbot_model.generate(input_ids=input_ids, max_length=200, do_sample=True, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Interactive chatbot
print("AI Assistant: Hello! I can help you identify plant diseases and answer related queries.")
current_disease = None

while True:
    if not current_disease:
        image_path = input("AI Assistant: Please provide the path to a plant image: ")
        if not os.path.exists(image_path):
            print("AI Assistant: Invalid path. Please try again.")
            continue

        predicted_disease = predict_disease(image_path)
        if "Error" in predicted_disease:
            print(f"AI Assistant: {predicted_disease}")
            continue

        current_disease = predicted_disease
        print(f"AI Assistant: The plant seems to have {current_disease}.")
        print("AI Assistant: You can ask about symptoms, treatments, or any other details. Type 'new' to analyze a new image.")
        continue

    user_query = input("You: ").strip().lower()
    if user_query == "new":
        current_disease = None
        print("AI Assistant: Let's analyze a new image!")
        continue

    response = get_chatbot_response(current_disease, user_query)
    print(f"AI Assistant: {response}")
"""

with open(script_path, "w") as f:
    f.write(code)

print(f"Chatbot script saved at {script_path}")


In [ ]:
!python /content/drive/MyDrive/chatbot.py

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Plant disease model loaded successfully.
2025-04-22 07:08:27.076140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745305707.096099    4162 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for p